### Import Modules ###

In [24]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import  linear_model
from pandas.tools.plotting import scatter_matrix
%matplotlib inline

### Loading the training data ###

In [25]:
train_df = pd.read_json('./data/train.json')
test_df = pd.read_json('./data/test.json')

In [26]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

### Create new Features ###
- num_photos
- num_features
- num_description_words
- created
- created_year
- created_month
- created_day


In [30]:
train_df["num_photos"] = train_df["photos"].apply(len)
test_df["num_photos"] = test_df["photos"].apply(len)

train_df["num_features"] = train_df["features"].apply(len)
test_df["num_features"] = test_df["features"].apply(len)

train_df["num_description_words"] = train_df["description"].apply(lambda x: len(x.split(" ")))
test_df["num_description_words"] = test_df["description"].apply(lambda x: len(x.split(" ")))

train_df["created"] = pd.to_datetime(train_df["created"])
test_df["created"] = pd.to_datetime(test_df["created"])

train_df["created_year"] = train_df["created"].dt.year
test_df["created_year"] = test_df["created"].dt.year

train_df["created_month"] = train_df["created"].dt.month
test_df["created_month"] = test_df["created"].dt.month

train_df["created_day"] = train_df["created"].dt.day
test_df["created_day"] = test_df["created"].dt.day

use_feats = ["bathrooms", "bedrooms", "latitude", "longitude", "price",
             "num_photos", "num_features", "num_description_words",
             "created_year", "created_month", "created_day"]


In [31]:
X_train = train_df
X_test = test_df

### K-means Clustering ###
longtitude 와 latitude 이용하여 K-means Clustering 시행.
cluster=10 , random_state=5 적용

In [33]:
from sklearn.cluster import KMeans

X_train['response'] = 0.
X_train.loc[X_train.interest_level=='medium', 'response'] = 0.5
X_train.loc[X_train.interest_level=='high', 'response'] = 1

print('Length before removing ouliers', len(X_train))
for i in ['latitude', 'longitude']:
    while(1):
        x = X_train[i].median()
        ix = abs(X_train[i] - x) > 3*X_train[i].std()
        if ix.sum()==0: # no more outliers -> stop
            break
        X_train.loc[ix, i] = np.nan # exclude outliers
# Keep only non-outlier listings
X_train = X_train.loc[X_train[['latitude', 'longitude']].isnull().sum(1) == 0, :]
print('Length after removing ouliers', len(X_train))

ncomp = [5,10,20,40]
r = KMeans(ncomp[1], random_state=5)
    # Normalize (longitude, latitude) before K-means
temp = X_train[['longitude', 'latitude']].copy()
temp['longitude'] = (temp['longitude']-temp['longitude'].mean())/temp['longitude'].std()
temp['latitude'] = (temp['latitude']-temp['latitude'].mean())/temp['latitude'].std()
    # Fit k-means and get labels
r.fit(temp[['longitude', 'latitude']])
X_train['labels'] = r.labels_

r.fit(X_test[['longitude', 'latitude']])
X_test['labels'] = r.labels_

use_feats.append('labels')

/home/seojayoung/miniconda3/envs/ml_python/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/home/seojayoung/miniconda3/envs/ml_python/lib/python3.5/site-packages/pandas/core/indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Length before removing ouliers 46946
Length after removing ouliers 46804


### New Feature ###
- Elevator
- Dogs Allowed
- Cata Allowed
- pet_frendly

In [34]:
def newfeat(name, df, series):
    """Create a Series for my feature building loop to fill"""
    feature = pd.Series(0, df.index, name=name)
    """Now populate the new Series with numeric values"""
    for row, word in enumerate(series):
        if name in word:
            feature.iloc[row] = 1
    df[name] = feature
    return(df)
   
X_train = newfeat('Elevator', X_train, X_train.features)
X_train = newfeat('Dogs Allowed', X_train, X_train.features)
X_train = newfeat('Cats Allowed', X_train, X_train.features)
X_train['pet_friendly'] = X_train['Cats Allowed'] + X_train['Dogs Allowed']

X_test = newfeat('Elevator', X_test, X_test.features)
X_test = newfeat('Dogs Allowed', X_test, X_test.features)
X_test = newfeat('Cats Allowed', X_test, X_test.features)
X_test['pet_friendly'] = X_test['Cats Allowed'] + X_test['Dogs Allowed']


### Feature Engineering –manager_id ###

manager_id가 받은 listing_id에 대한interest_level(low, medium, high)을count 한 후 한 manager_id가 받은 전체평가 개수로나눠 준다.
manager_level_low, manager_level_medium, manager_level_high추가

In [36]:
import random

index=list(range(X_train.shape[0]))
random.shuffle(index)
a=[np.nan]*len(X_train)
b=[np.nan]*len(X_train)
c=[np.nan]*len(X_train)

for i in range(5):
    building_level={}
    for j in X_train['manager_id'].values:
        building_level[j]=[0,0,0]
    test_index=index[int((i*X_train.shape[0])/5):int(((i+1)*X_train.shape[0])/5)]
    train_index=list(set(index).difference(test_index))
    for j in train_index:
        temp=X_train.iloc[j]
        if temp['interest_level']=='low':
            building_level[temp['manager_id']][0]+=1
        if temp['interest_level']=='medium':
            building_level[temp['manager_id']][1]+=1
        if temp['interest_level']=='high':
            building_level[temp['manager_id']][2]+=1
    for j in test_index:
        temp=X_train.iloc[j]
        if sum(building_level[temp['manager_id']])!=0:
            a[j]=building_level[temp['manager_id']][0]*1.0/sum(building_level[temp['manager_id']])
            b[j]=building_level[temp['manager_id']][1]*1.0/sum(building_level[temp['manager_id']])
            c[j]=building_level[temp['manager_id']][2]*1.0/sum(building_level[temp['manager_id']])
X_train['manager_level_low']=a
X_train['manager_level_medium']=b
X_train['manager_level_high']=c

a=[]
b=[]
c=[]
building_level={}
for j in X_train['manager_id'].values:
    building_level[j]=[0,0,0]
for j in range(X_train.shape[0]):
    temp=X_train.iloc[j]
    if temp['interest_level']=='low':
        building_level[temp['manager_id']][0]+=1
    if temp['interest_level']=='medium':
        building_level[temp['manager_id']][1]+=1
    if temp['interest_level']=='high':
        building_level[temp['manager_id']][2]+=1

for i in X_test['manager_id'].values:
    if i not in building_level.keys():
        a.append(np.nan)
        b.append(np.nan)
        c.append(np.nan)
    else:
        a.append(building_level[i][0]*1.0/sum(building_level[i]))
        b.append(building_level[i][1]*1.0/sum(building_level[i]))
        c.append(building_level[i][2]*1.0/sum(building_level[i]))

X_test['manager_level_low']=a
X_test['manager_level_medium']=b
X_test['manager_level_high']=c

### nan 값 변환 ###
X_train , X_test의 타입을 float32로 바꿔준 다음,
nan 값을 0.0 으로 변환

In [37]:
X_train[['manager_level_low', 'manager_level_medium', 'manager_level_high']] = X_train[['manager_level_low', 'manager_level_medium', 'manager_level_high']].astype(np.float32)
X_test[['manager_level_low', 'manager_level_medium', 'manager_level_high']] = X_test[['manager_level_low', 'manager_level_medium', 'manager_level_high']].astype(np.float32)

X_train[['manager_level_low', 'manager_level_medium', 'manager_level_high']] = X_train[['manager_level_low', 'manager_level_medium', 'manager_level_high']].replace(np.nan, 0.0, regex=True)
X_test[['manager_level_low', 'manager_level_medium', 'manager_level_high']] = X_test[['manager_level_low', 'manager_level_medium', 'manager_level_high']].replace(np.nan, 0.0, regex=True)

### Feature 추가 ###
use_feats 변수에 manager_level_low, manager_level_medium, manager_level_high 변수를 추가

In [38]:
use_feats.append('manager_level_low') 
use_feats.append('manager_level_medium') 
use_feats.append('manager_level_high')

### XGBoost ###

In [39]:
from scipy import sparse
import xgboost as xgb
import random
from sklearn import model_selection, preprocessing, ensemble
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

target_num_map = {'high':0, 'medium':1, 'low':2}
train_y = np.array(X_train['interest_level'].apply(lambda x: target_num_map[x]))

train_test = sparse.coo_matrix(X_train[use_feats]).tocsr()
test_test = sparse.coo_matrix(X_test[use_feats]).tocsr()

def runXGB(train_X, train_y, test_X, test_y=None, feature_names=None, seed_val=0, num_rounds=1000):
    param = {}
    param['objective'] = 'multi:softprob'
    param['eta'] = 0.03
    param['max_depth'] = 6
    param['silent'] = 1
    param['num_class'] = 3
    param['eval_metric'] = "mlogloss"
    param['min_child_weight'] = 1
    param['subsample'] = 0.7
    param['colsample_bytree'] = 0.7
    param['seed'] = seed_val
    num_rounds = num_rounds

    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y)
        watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=20)
    else:
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst, xgtrain, num_rounds)

    pred_test_y = model.predict(xgtest)
    return pred_test_y, model

In [40]:
cv_scores = []
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=2016)
for dev_index, val_index in kf.split(range(train_test.shape[0])):
        dev_X, val_X = train_test[dev_index,:], train_test[val_index,:]
        dev_y, val_y = train_y[dev_index], train_y[val_index]
        preds, model = runXGB(dev_X, dev_y, val_X, val_y)
        cv_scores.append(log_loss(val_y, preds))
        print(cv_scores)
        break

preds, model = runXGB(train_test, train_y, test_test, num_rounds=1000)

[0]	train-mlogloss:1.07785	test-mlogloss:1.07822
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 20 rounds.
[1]	train-mlogloss:1.05857	test-mlogloss:1.05928
[2]	train-mlogloss:1.04109	test-mlogloss:1.04201
[3]	train-mlogloss:1.02358	test-mlogloss:1.02487
[4]	train-mlogloss:1.00686	test-mlogloss:1.00844
[5]	train-mlogloss:0.991118	test-mlogloss:0.992947
[6]	train-mlogloss:0.9764	test-mlogloss:0.978528
[7]	train-mlogloss:0.961604	test-mlogloss:0.964009
[8]	train-mlogloss:0.947803	test-mlogloss:0.950431
[9]	train-mlogloss:0.934412	test-mlogloss:0.937357
[10]	train-mlogloss:0.921784	test-mlogloss:0.924965
[11]	train-mlogloss:0.909799	test-mlogloss:0.913268
[12]	train-mlogloss:0.898283	test-mlogloss:0.902009
[13]	train-mlogloss:0.886959	test-mlogloss:0.891004
[14]	train-mlogloss:0.876455	test-mlogloss:0.880756
[15]	train-mlogloss:0.865978	test-mlogloss:0.870634
[16]	train-mlogloss:0.856037	test-mlogl

- train-mlogloss : 0.397172 
- test-mlogloss  : 0.550993

In [41]:
out_df = pd.DataFrame(preds)
out_df.columns = ["high", "medium", "low"]
out_df["listing_id"] = X_test.listing_id.values

out_df.to_csv("submission_4.csv", index=False)

*실제 제출 시 Score = 0.53328*